# Session 1: Qubit-level benchmarks & measurement error mitigation

This notebook gives examples for how to use the ``ignis.characterization.coherence`` module for measuring $T_1$ and $T_2$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import qiskit
from qiskit.providers.aer.noise.errors.standard_errors import thermal_relaxation_error
from qiskit.providers.aer.noise import NoiseModel

from qiskit.ignis.characterization.coherence import T1Fitter, T2StarFitter, T2Fitter
from qiskit.ignis.characterization.coherence import t1_circuits, t2_circuits, t2star_circuits

To estimate $T_{1}$ and $T_{2}$ times, we need to run certain structured circuits (an "experiment design"). Qiskit's `characterization` module provides easy helper functions for generating such circuits.

# Estimating $T_{1}$ time

## Generate the $T_1$ experiment design

In a $T_{1}$ experiment, we start with a qubit in the $|0\rangle$ state, and then excite the qubit to the $|1\rangle$ state using and $X$-gate. We then wait some number of time steps (delays), and measure the probability the qubit is still in the $|1\rangle$ state.

Below, we set the delay to be some number of identity gates (``iden``); the gate time for an identity gate is available from the backend being characterized. Here, we hard-code it as .1 micro-seconds.

Because the $T_1$ time is measured from a fit of the results of circuits with _varying_ delays, we have to specify the minimum and maximum delays.

In [ ]:
# Set the number of identity gates to be used in the circuits.
# Here, the minimum number of gates is 10, and the maximum, 300.
# We step in units of 50.
num_of_gates = (np.linspace(10, 300, 50)).astype(int)

# We hard-code the gate time for the identity as .1 micro-seconds.
gate_time = 0.1

# We specify which qubit(s) we want to measure T1 times for.
# Note that it is possible to measure several qubits in parallel
qubits = [0]

The function `t1_circuits` will return both the circuits and delay information. The former we run on the hardware or simulator, and the latter, we use later for fitting purposes.

In [ ]:
t1_circs, t1_xdata = t1_circuits(num_of_gates, gate_time, qubits)

Let's take a look at the first and last circuits. Note that _barriers_ are inserted into the circuits to prevent Qiskit's transpiler from collapsing the identity gates.

In [ ]:
# As expected, there are 10 identity gates, separated by barriers.
t1_circs[0].draw()

In [ ]:
t1_circs[-1].draw()

In addition to the circuits in the experiment design, `t1_circuits` also returns the delay times. These are calculated as (number of identity gates) * (time per identity gate).

In [ ]:
# For the first circuit, there are 10 identities, so the delay time is (.1)*10 = 1 micro-second
t1_xdata

## Simulate running a T1 experiment

### Building a noise model

To get an intuition for what the $T_{1}$ time is, and how properties of the hardware affect it, we'll use a noisy simulation to mock up running the experiment design on real hardware.

We'll assume the hardware has  a $T_1$ time of 25 micro-seconds.

In [ ]:
t1_true = 25.0

# Instantiate the noise model
t1_noise_model = NoiseModel()

# Add an error corresponding to thermal relaxation
# which acts on the identity gate.
# In the noise model, we have to specify which qubit(s) the noise
# acts on.
# We also set the T2 time of the qubits in the model to be the maximum allowed
# by the T1 time; namely, 2*T1
t1_noise_model.add_quantum_error(
    thermal_relaxation_error(t1_true, 2*t1_true, gate_time), 
    'id', [0])

### Run the simulation

In [ ]:
# Run the simulator, where we mock up repeating each circuit
# in the experiment design 500 times ("shots").
backend = qiskit.Aer.get_backend('qasm_simulator')
shots = 500

t1_backend_result = qiskit.execute(t1_circs, backend, shots=shots,
                                   noise_model=t1_noise_model, optimization_level=0).result()

The `t1_backend_result` contains the results("counts") from running the circuits in the simulator. We can access the results associated with the $j^{th}$ circuit in the experiment design by using `.get_counts(j)`.

In [ ]:
# Look at results for the first and last circuits  in the experiment design.
t1_backend_result.get_counts(0), t1_backend_result.get_counts(-1)

For the first circuit, most of the counts are `1`, meaning the qubit survived. But for the last circuit, most are `0`, meaning it decayed.

We estimate the rate of decay to get an estimate of the $T_{1}$ time.

### Fit the data

We estimate the $T_{1}$ time by fitting the data of delay times and survival probabilities to the following model:
$$ \mathrm{Pr(survival)} = Ae^{-t/T_{1}} + B.$$

The fit to this model is done using the `T1Fitter` function. This function includes keyword arguments which ask us to specify initial guesses and bounds on $(A,T_{1}, B)$.

In [ ]:
# The ordering of the guesses and bounds goes (A, T_{1}, B)
t1_fit = T1Fitter(t1_backend_result, t1_xdata, qubits,
                  fit_p0=[1, t1_true, 0],
                  fit_bounds=([0, 0, -1], [2, 40, 1]))

From this fit object, we can extract several quantities of interest, including an estimated $T_{1}$ time.

First though, let's plot the data to see what's going on.

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

# Specify we want the decay curve for qubit 0.
t1_fit.plot(0, ax=ax)

As the delay time increases, the probability the qubit remains in the $|1\rangle$ state goes down.

The data is fit to an exponential curve, and the coefficient of decay is the estimated $T_{1}$ time.

Let's check what the fit parameters are:

In [ ]:
print('Estimated A, T_1, B:', t1_fit.params)
print('Error bars on the estimates:', t1_fit.params_err)

If we wanted just the $T_{1}$ time and error, we can use some convenience functions:

In [ ]:
print('Estimated $T_{1}$ time (micro-seconds):', t1_fit.time()[0])
print('Error bars on the estimate (micro-seconds):', t1_fit.time_err()[0])


# Estimate $T_{2}$ time 

## Generate a $T_{2}$ experiment design

There are several different kinds of $T_{2}$ experiment designs, which we look at below.

The core idea of the experiment design is to measure dephasing noise affecting a qubit.

Below, we set the delay to be some number of identity gates (``iden``); the gate time for an identity gate is available from the backend being characterized. Here, we hard-code it as .1 micro-seconds.

Because the $T_2$ time is measured from a fit of the results of circuits with _varying_ delays, we have to specify the minimum and maximum delays.

In [ ]:
# Set the number of identity gates to be used in the circuits.
# Here, the minimum number of gates is 10, and the maximum, 300.
# We step in units of 50.
num_of_gates = (np.linspace(5, 150, 25)).astype(int)

# We hard-code the gate time for the identity as .1 micro-seconds.
gate_time = 0.1

# We specify which qubit(s) we want to measure T2 times for.
# Note that it is possible to measure several qubits in parallel
qubits = [0]

In [ ]:
t2echo_circs, t2echo_xdata = t2_circuits(num_of_gates,
                                         gate_time, qubits, n_echos=1, phase_alt_echo=False)

In [ ]:
t2echo_circs[0].draw()

### A slightly more sophisticated experiment design using an $RZ$ gate

In [ ]:
t2star_circs, t2star_xdata, osc_freq = t2star_circuits(num_of_gates, gate_time, qubits, nosc=5)

In [ ]:
t2star_circs[0].draw()

### An even more sophisticated experiment design using CPMG-based circuits

In [ ]:
t2cpmg_circs, t2cpmg_xdata = t2_circuits(num_of_gates, 
                                         gate_time, qubits, 
                                         n_echos=1, phase_alt_echo=True)

In [ ]:
t2cpmg_circs[0].draw()

## Simulate Running a $T_{2}$ experiment

We'll use a simulator, and a noise model, to mock up running these circuits on hardware.

### Building a noise model

In [ ]:
# Assume the true T2 time is 25 micro-seconds
t2_true = 25.0


t2_noise_model = NoiseModel()

# Notice that here we assume the T1 time is infinite,
# so that contributions to decays caused by T1 processes
# is zero.
t2_noise_model.add_quantum_error(
    thermal_relaxation_error(np.inf, t2_true, gate_time, 0.5), 
    'id', [0])

## Run the simulation

In [ ]:
# Run the simulator, where we mock up repeating each circuit
# in the experiment design 500 times ("shots").
backend = qiskit.Aer.get_backend('qasm_simulator')
shots = 500

t2star_backend_result = qiskit.execute(t2star_circs, backend, shots=shots,
                                       noise_model=t2_noise_model, optimization_level=0).result()
t2echo_backend_result = qiskit.execute(t2echo_circs, backend, shots=shots,
                                       noise_model=t2_noise_model, optimization_level=0).result()
t2cpmg_backend_result = qiskit.execute(t2cpmg_circs, backend,
                                        shots=shots, noise_model=t2_noise_model,
                                        optimization_level=0).result()

The `*_backend_result` objects contain the results("counts") from running the circuits in the simulator. We can access the results associated with the $j^{th}$ circuit in the experiment design by using `.get_counts(j)`.

In [ ]:
t2star_backend_result.get_counts(0), t2echo_backend_result.get_counts(0), t2cpmg_backend_result.get_counts(0)

Note that each experiment design has a different structure to the circuits, so getting different counts is to be expected.

### Fit the data

#### $T_{2}^{\star}$ time

For estimating the $T_{2}^{\star}$ time, the data is fit to the model

$$ \mathrm{Pr(qubit~is~0)} = 2Ae^{-t/T_{2}^{\star}}\cos(2\pi f t + \phi) + B$$

In [ ]:
t2star_fit = T2StarFitter(t2star_backend_result, t2star_xdata, qubits,
                          fit_p0=[0.5, t2_true, osc_freq, 0, 0.5],
                          fit_bounds=([-0.5, 0, 0, -np.pi, -0.5],
                                      [1.5, 40, 2*osc_freq, np.pi, 1.5]))

Let's plot the data and the fit.

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

# Specify we want the decay curve for qubit 0.
t2star_fit.plot(0, ax=ax)

Let's check the fit parameters:

In [ ]:
print('Estimated (A, T2*, f, phi, B):', t2star_fit.params)
print('Error bars on the estimates:', t2star_fit.params_err)

And if we're only interested in the estimated $T_{2}^{\star}$ time (and its error), we use the `.time()` and `

In [ ]:
print('Estimated T2* time (micro-seconds):', t2star_fit.time())
print('Estimated error bars (micro-seconds):', t2star_fit.time_err())

#### $T_{2}$ time

For estimating the $T_{2}$ time, the data is fit to the model

$$ \mathrm{Pr(qubit~is~0)} = Ae^{-t/T_{2}} + B$$

Here, we can use both the echoed and CPMG circuits to see how the experiment design impacts the estimate.

In [ ]:
t2echo_fit = T2Fitter(t2echo_backend_result, t2echo_xdata, qubits,
                      fit_p0=[0.5, t2_true, 0.5],
                      fit_bounds=([-0.5, 0, -0.5],
                                  [1.5, 40, 1.5]))

t2cpmg_fit = T2Fitter(t2cpmg_backend_result,
                      t2cpmg_xdata, qubits,
                      fit_p0=[0.5, t2_true, 0.5],
                      fit_bounds=([-0.5, 0, -0.5],
                                  [1.5, 40, 1.5]))

Let's plot the data.

In [ ]:
fig = plt.figure(figsize=(15, 5))

ax = fig.add_subplot(1, 2, 1)
t2echo_fit.plot(0, ax=ax)
ax.set_title('Echoed circuits', fontsize=15)

ax = fig.add_subplot(1, 2, 2)
t2cpmg_fit.plot(0, ax=ax)
ax.set_title('CPMG circuits', fontsize=15)

The data are similar, but there is variation.

In [ ]:
print('Using the echoed circuits, the estimated $T_{2}$ time is:', t2echo_fit.time())
print('The error bars are estimated to be ', t2echo_fit.time_err())
print('')

print('Using the echoed circuits, the estimated $T_{2}$ time is:', t2echo_fit.time())
print('The error bars are estimated to be ', t2echo_fit.time_err())

# Measurement Error Mitigation

## Introduction

The measurement calibration is used to mitigate measurement errors. From these calibrations, it is possible to correct the average results of another experiment of interest. This notebook gives examples for how to use the ``ignis.mitigation.measurement`` module.

Here, we will do a "complete" measurement calibration;  for more details on this and other measurement error mitigation techniques, see [this chapter](https://qiskit.org/textbook/ch-quantum-hardware/measurement-error-mitigation.html) in the Qiskit textbook.

In [ ]:
# Import general libraries (needed for functions)
import numpy as np
import time

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.providers.aer import noise
from qiskit.tools.visualization import plot_histogram

# Import measurement calibration functions
from qiskit.ignis.mitigation.measurement import (complete_meas_cal,
                                                 CompleteMeasFitter)

Assume that we would like to generate a calibration matrix for the 3 qubits Q0, Q1 and Q2 in a 3-qubit Quantum Register [Q0,Q1,Q2]. 

Since we have 3 qubits, there are $2^3=8$ possible quantum states.

## Generating Measurement Calibration Circuits

First, we generate a list of measurement calibration circuits for the full Hilbert space. Each circuit creates a basis state.  If there are $n=3$ qubits, then we get $2^3=8$ calibration circuits.

The following function **complete_meas_cal** returns a list **meas_calibs** of `QuantumCircuit` objects containing the calibration circuits, 
and a list **state_labels** of the calibration state labels.

The input to this function can be given in one of the following three forms:

- **qubit_list:** A list of qubits to perform the measurement correction on, or:
- **qr (QuantumRegister):** A quantum register, or:
- **cr (ClassicalRegister):** A classical register.

In addition, one can provide a string **circlabel**, which is added at the beginning of the circuit names for unique identification.

In [ ]:
# Generate the calibration circuits
qr = qiskit.QuantumRegister(3)
qubit_list = [0,1,2]
meas_calibs, state_labels = complete_meas_cal(qubit_list=qubit_list, qr=qr, circlabel='my_calibration')

In [ ]:
# Look at a measurement calibration circuit.
# This circuit simply prepares the |000> state,
# and then immediately measures the qubits.
meas_calibs[0].draw()

In [ ]:
# Look at a measurement calibration circuit.
# This circuit simply prepares the |111> state,
# and then immediately measures the qubits.
meas_calibs[-1].draw()

## Computing the Calibration Matrix

If we do not apply any noise, then the calibration matrix is expected to be the $8 \times 8$ identity matrix.

In [ ]:
# Execute the calibration circuits without noise
backend = qiskit.Aer.get_backend('qasm_simulator')
job = qiskit.execute(meas_calibs, backend=backend, shots=1000)
cal_results = job.result()

In [ ]:
# The calibration matrix without noise is the identity matrix
meas_fitter = CompleteMeasFitter(cal_results, state_labels)
print(meas_fitter.cal_matrix)

Assume that we apply some noise model from Qiskit Aer to the 3 qubits, 
then the calibration matrix will have most of its mass on the main diagonal, with some additional 'noise'.

Alternatively, we can execute the calibration circuits on real quantum systems using the IBMQ provider.

In [ ]:
# Generate a noise model for the 3 qubits.
# We will use a model wherein, for each qubit,
# the following are the probabilities of measurement outcomes:
# Pr("0" | |0>) = .9 & Pr("1" | |1>) = .75

# Encode measurement probabilities for |0> state
readout_error0 = [.9, .1]

# Encode measurement probabilities for |1> state
readout_error1 = [.25, .75]

noise_model = noise.NoiseModel()
for qi in range(3):
    read_err = noise.errors.readout_error.ReadoutError([readout_error0, readout_error1])
    noise_model.add_readout_error(read_err, [qi])

In [ ]:
# Execute the calibration circuits
backend = qiskit.Aer.get_backend('qasm_simulator')
job = qiskit.execute(meas_calibs, backend=backend, shots=1000, noise_model=noise_model)
cal_results = job.result()

In [ ]:
# Calculate the calibration matrix with the noise model
meas_fitter = CompleteMeasFitter(cal_results, state_labels, qubit_list=qubit_list)
print(meas_fitter.cal_matrix)

In [ ]:
# Let's plot it, for easier visualization
meas_fitter.plot_calibration()

Most of the weight of the entries of this matrix is along the diagonal (meaning little mis-assignment of the states of the qubits), but there is some weight in the off-diagonals. So sometimes, when we have prepared a state, we'll mis-assign it.

## Analyzing the Results

We would like to compute the total measurement fidelity, and the measurement fidelity for a specific qubit, for example, Q0.

Since the on-diagonal elements of the calibration matrix are the probabilities of measuring state $j$ given preparation of state $j$, 
then the trace of this matrix is the average assignment fidelity:

$$\mathrm{Pr(~successful~assignment)} = \sum_{j}\mathrm{Pr}(j|j)$$

In [ ]:
# What is the measurement fidelity?
print("Average Measurement Fidelity: %f" % meas_fitter.readout_fidelity())

## Applying the Calibration

We now perform another experiment and correct the measured results. 

## Correct Measurement Noise on a 3Q GHZ State

As an example, let's prepare a 3-qubit GHZ state:

$$ \mid GHZ \rangle = \frac{\mid{000} \rangle + \mid{111} \rangle}{\sqrt{2}}$$

In [ ]:
# Make a 3Q GHZ state
ghz = QuantumCircuit(qr)
ghz.h(qr[0])
ghz.cx(qr[0], qr[1])
ghz.cx(qr[0], qr[2])
ghz.measure_all()

In [ ]:
# Draw the circuit, so we know what we're dealing with.
ghz.draw(output='mpl')

We now execute the calibration circuits (with the same noise model as above):

In [ ]:
job = qiskit.execute([ghz], backend=backend, shots=5000, noise_model=noise_model)
results = job.result()

We now compute the results without any error mitigation and with the mitigation, namely after applying the calibration matrix to the results.

There are two fitting methods for applying the calibration (if no method is defined, then 'least_squares' is used). 
- **'pseudo_inverse'**, which is a direct inversion of the calibration matrix, 
- **'least_squares'**, which constrains to have physical probabilities.

The raw data to be corrected can be given in a number of forms:

- Form1: A counts dictionary from results.get_counts,
- Form2: A list of counts of length=len(state_labels),
- Form3: A list of counts of length=M*len(state_labels) where M is an integer (e.g. for use with the tomography data),
- Form4: A qiskit Result (e.g. results as above).

In [ ]:
# Results without mitigation
raw_counts = results.get_counts()

# Get the filter object
meas_filter = meas_fitter.filter

# Results with mitigation
mitigated_results = meas_filter.apply(results)
mitigated_counts = mitigated_results.get_counts(0)

Let's plot the raw and mitigated counts.

For an ideal GHZ state, we should see `000` and `111` both with probability 1/2.

In [ ]:
from qiskit.tools.visualization import *
plot_histogram([raw_counts, mitigated_counts], legend=['raw', 'mitigated'])

The mitigated counts are much closer to the ideal!

## Applying to a reduced subset of qubits

Consider now that we want to correct a 2-qubit Bell state, but we have the 3-qubit calibration matrix. We can reduce the matrix and build a new mitigation object.

In [ ]:
# Make a 2Q Bell state between Q0 and Q2
cr = ClassicalRegister(2)
bell = QuantumCircuit(qr, cr)
bell.h(qr[0])
bell.cx(qr[0], qr[2])
bell.measure(qr[0],cr[0])
bell.measure(qr[2],cr[1])

In [ ]:
# Let's draw the circuit, so we know what we're working with.
bell.draw(output='mpl')

We'll simulate running this circuit on HW using the same noise model as we did for the GHZ state.

In [ ]:
job = qiskit.execute([bell], backend=backend, shots=5000, noise_model=noise_model)
results = job.result()

Now, to mitigate these results, we need to reduce the calibration matrix to one which only acts on qubits 0 and 2.

In [ ]:
#build a fitter from the subset
meas_fitter_sub = meas_fitter.subset_fitter(qubit_sublist=[0,2])

In [ ]:
#The calibration matrix is now in the space Q0/Q2
meas_fitter_sub.cal_matrix

Let's plot the raw and mitigated counts.

For an ideal Bell state, we should see `00` and `11` with probability 1/2.

In [ ]:
# Results without mitigation
raw_counts = results.get_counts()

# Get the filter object
meas_filter_sub = meas_fitter_sub.filter

# Results with mitigation
mitigated_results = meas_filter_sub.apply(results)
mitigated_counts = mitigated_results.get_counts(0)
from qiskit.tools.visualization import *
plot_histogram([raw_counts, mitigated_counts], legend=['raw', 'mitigated'])

Again, the mitigated results are in much closer agreement to the ideal probabilities.